# OpenJij Benchmark Visualizations

This notebook generates high-resolution visualizations from Phase 1 and Phase 2 benchmark results.

**Input Files:**
- `data/results/phase1_openjij_parameters.xlsx` - QUBO parameter optimization results
- `data/results/phase2_openjij_parameters.xlsx` - OpenJij annealing parameter optimization results

## Setup: Imports and Configuration

In [ ]:
# Add project root to Python path
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"✓ Project root: {project_root}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set high-quality plotting defaults
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 12
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.markersize'] = 8
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['grid.linewidth'] = 1.5
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['axes.labelsize'] = 13
plt.rcParams['axes.titlesize'] = 14

sns.set_style('whitegrid')

print("✓ All libraries imported successfully")
print("✓ High-resolution plotting configured (DPI=300, thick lines)")

## Load Phase 1 and Phase 2 Results

In [ ]:
# Define paths
results_dir = project_root / 'data' / 'results'
phase1_file = results_dir / 'phase1_openjij_parameters.xlsx'
phase2_file = results_dir / 'phase2_openjij_parameters.xlsx'
output_dir = results_dir / 'visualizations'
output_dir.mkdir(parents=True, exist_ok=True)

print("Loading benchmark results...")

# Load Phase 1 results
if not phase1_file.exists():
    raise FileNotFoundError(f"Phase 1 file not found: {phase1_file}")
phase1_df = pd.read_excel(phase1_file)
print(f"✓ Loaded Phase 1 results: {len(phase1_df)} configurations")
print(f"  Columns: {phase1_df.columns.tolist()}")

# Load Phase 2 results
if not phase2_file.exists():
    raise FileNotFoundError(f"Phase 2 file not found: {phase2_file}")
phase2_df = pd.read_excel(phase2_file)
print(f"✓ Loaded Phase 2 results: {len(phase2_df)} configurations")
print(f"  Columns: {phase2_df.columns.tolist()}")

# Extract parameter values for Phase 1
k_values = sorted(phase1_df['k'].unique())
alpha_values = sorted(phase1_df['alpha'].unique())
penalty_values = sorted(phase1_df['penalty'].unique())

print(f"\nPhase 1 parameter ranges:")
print(f"  k: {k_values}")
print(f"  alpha: {alpha_values}")
print(f"  penalty: {penalty_values}")

# Extract parameter values for Phase 2
num_sweeps_values = sorted(phase2_df['num_sweeps'].unique())
num_reads_values = sorted(phase2_df['num_reads'].unique())
beta_values = sorted(phase2_df['beta'].unique())
gamma_values = sorted(phase2_df['gamma'].unique())

print(f"\nPhase 2 parameter ranges:")
print(f"  num_sweeps: {num_sweeps_values}")
print(f"  num_reads: {num_reads_values}")
print(f"  beta: {beta_values}")
print(f"  gamma: {gamma_values}")

print(f"\n✓ Output directory: {output_dir}")

---
# Phase 1 Visualizations: QUBO Parameter Optimization

## Figure 1: Mean 3D Error vs Number of APs (k) for Different Alpha Values

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for alpha in alpha_values:
    subset = phase1_df[phase1_df['alpha'] == alpha]
    ax.plot(subset['k'], subset['mean_3d_error_m'], 
            marker='o', label=f'α={alpha}', linewidth=3)

ax.set_xlabel('k (Number of APs)', fontsize=14, fontweight='bold')
ax.set_ylabel('Mean 3D Error (m)', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'phase1_fig1_error_vs_k.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 1 saved: phase1_fig1_error_vs_k.png")

## Figure 2: Floor Accuracy vs Alpha for Different k Values

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for k_val in k_values:
    subset = phase1_df[phase1_df['k'] == k_val]
    ax.plot(subset['alpha'], subset['floor_accuracy_0'], 
            marker='s', label=f'k={k_val}', linewidth=3)

ax.set_xlabel('Alpha (α)', fontsize=14, fontweight='bold')
ax.set_ylabel('Floor Accuracy (exact)', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'phase1_fig2_floor_acc_vs_alpha.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 2 saved: phase1_fig2_floor_acc_vs_alpha.png")

## Figure 3: Mean 3D Error Heatmap (k vs Alpha)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

pivot_table = phase1_df.pivot_table(
    values='mean_3d_error_m', 
    index='k', 
    columns='alpha', 
    aggfunc='mean'
)

sns.heatmap(pivot_table, annot=True, fmt='.2f', cmap='RdYlGn_r', 
            ax=ax, cbar_kws={'label': 'Mean 3D Error (m)'}, 
            linewidths=2, linecolor='white',
            annot_kws={'fontsize': 11, 'fontweight': 'bold'})

ax.set_xlabel('Alpha (α)', fontsize=14, fontweight='bold')
ax.set_ylabel('k (Number of APs)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(output_dir / 'phase1_fig3_heatmap_k_alpha.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 3 saved: phase1_fig3_heatmap_k_alpha.png")

## Figure 4: Positioning Accuracy vs Floor Accuracy (Penalty Effect)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for penalty_val in penalty_values:
    subset = phase1_df[phase1_df['penalty'] == penalty_val]
    ax.scatter(subset['mean_3d_error_m'], subset['floor_accuracy_0'], 
               label=f'penalty={penalty_val}', s=200, alpha=0.7, 
               edgecolors='black', linewidths=1.5)

ax.set_xlabel('Mean 3D Error (m)', fontsize=14, fontweight='bold')
ax.set_ylabel('Floor Accuracy (exact)', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'phase1_fig4_penalty_effect.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 4 saved: phase1_fig4_penalty_effect.png")

---
# Phase 2 Visualizations: OpenJij Annealing Parameter Optimization

In [ ]:
# Filter out infinite TTS values for visualization
phase2_finite = phase2_df[phase2_df['tts_s'] != float('inf')].copy()

# Find best configuration for marking
phase2_sorted = phase2_finite.sort_values('tts_s')
best_config = phase2_sorted.iloc[0]

print(f"Best OpenJij configuration (lowest TTS):")
print(f"  num_sweeps: {int(best_config['num_sweeps'])}")
print(f"  num_reads: {int(best_config['num_reads'])}")
print(f"  beta: {best_config['beta']}")
print(f"  gamma: {best_config['gamma']}")
print(f"  TTS: {best_config['tts_s']:.3f}s")
print(f"  Floor Accuracy: {best_config['floor_accuracy_0']:.2%}")

## Figure 5: Time-to-Solution vs Number of Sweeps for Different num_reads

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for num_reads_val in num_reads_values:
    subset = phase2_finite[phase2_finite['num_reads'] == num_reads_val]
    # Group by num_sweeps and take mean of TTS
    grouped = subset.groupby('num_sweeps')['tts_s'].mean().reset_index()
    ax.plot(grouped['num_sweeps'], grouped['tts_s'], 
            marker='o', label=f'reads={num_reads_val}', linewidth=3)

ax.set_xlabel('Number of Sweeps', fontsize=14, fontweight='bold')
ax.set_ylabel('Time-to-Solution (s)', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.legend(fontsize=12, frameon=True, shadow=True)
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig(output_dir / 'phase2_fig5_tts_vs_sweeps.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 5 saved: phase2_fig5_tts_vs_sweeps.png")

## Figure 6: Success Rate vs Number of Reads

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

ax.scatter(phase2_finite['num_reads'], phase2_finite['success_rate'], 
           alpha=0.6, s=200, edgecolors='black', linewidths=1.5)

ax.set_xlabel('Number of Reads', fontsize=14, fontweight='bold')
ax.set_ylabel('Success Rate', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'phase2_fig6_success_vs_reads.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 6 saved: phase2_fig6_success_vs_reads.png")

## Figure 7: Time-to-Solution vs Beta (Inverse Temperature) for Different Gamma Values

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for gamma_val in gamma_values:
    subset = phase2_finite[phase2_finite['gamma'] == gamma_val]
    # Group by beta and take mean of TTS
    grouped = subset.groupby('beta')['tts_s'].mean().reset_index()
    ax.plot(grouped['beta'], grouped['tts_s'], 
            marker='s', label=f'γ={gamma_val}', linewidth=3)

ax.set_xlabel('Beta (Inverse Temperature)', fontsize=14, fontweight='bold')
ax.set_ylabel('Time-to-Solution (s)', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.legend(fontsize=12, frameon=True, shadow=True)
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig(output_dir / 'phase2_fig7_tts_vs_beta.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 7 saved: phase2_fig7_tts_vs_beta.png")

## Figure 8: Floor Accuracy vs Time-to-Solution (Colored by num_sweeps)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

scatter = ax.scatter(phase2_finite['tts_s'], phase2_finite['floor_accuracy_0'], 
                     c=phase2_finite['num_sweeps'], cmap='viridis', 
                     s=200, alpha=0.7, edgecolors='black', linewidths=1.5)

ax.set_xlabel('Time-to-Solution (s)', fontsize=14, fontweight='bold')
ax.set_ylabel('Floor Accuracy (exact)', fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.grid(True, alpha=0.3)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('num_sweeps', fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

plt.tight_layout()
plt.savefig(output_dir / 'phase2_fig8_floor_acc_vs_tts.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 8 saved: phase2_fig8_floor_acc_vs_tts.png")

## Figure 9: Mean 3D Error vs Time-to-Solution (Colored by Success Rate)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))

scatter = ax.scatter(phase2_finite['tts_s'], phase2_finite['mean_3d_error_m'], 
                     c=phase2_finite['success_rate'], cmap='RdYlGn', 
                     s=200, alpha=0.7, edgecolors='black', linewidths=1.5,
                     vmin=0, vmax=1)

ax.set_xlabel('Time-to-Solution (s)', fontsize=14, fontweight='bold')
ax.set_ylabel('Mean 3D Error (m)', fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.grid(True, alpha=0.3)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Success Rate', fontsize=13, fontweight='bold')
cbar.ax.tick_params(labelsize=11)

plt.tight_layout()
plt.savefig(output_dir / 'phase2_fig9_error_vs_tts.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 9 saved: phase2_fig9_error_vs_tts.png")

## Figure 10: Optimization Trade-off (TTS vs Floor Accuracy with Best Configuration Marked)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

# Plot all configurations
ax.scatter(phase2_finite['tts_s'], phase2_finite['floor_accuracy_0'], 
           s=200, alpha=0.6, edgecolors='black', linewidths=1.5,
           label='All Configurations', color='steelblue')

# Mark best configuration
ax.scatter(best_config['tts_s'], best_config['floor_accuracy_0'], 
           s=500, color='red', marker='*', edgecolors='black', 
           linewidths=2.5, label='Best Configuration', zorder=10)

ax.set_xlabel('Time-to-Solution (s)', fontsize=14, fontweight='bold')
ax.set_ylabel('Floor Accuracy (exact)', fontsize=14, fontweight='bold')
ax.set_xscale('log')
ax.legend(fontsize=12, frameon=True, shadow=True)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'phase2_fig10_pareto_front.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Figure 10 saved: phase2_fig10_pareto_front.png")

---
## Summary

In [ ]:
print("="*80)
print("VISUALIZATION GENERATION COMPLETE")
print("="*80)
print(f"\nPhase 1 Figures (4 total):")
print(f"  1. Mean 3D Error vs k (different alphas)")
print(f"  2. Floor Accuracy vs alpha (different k values)")
print(f"  3. Heatmap of Mean 3D Error (k vs alpha)")
print(f"  4. Penalty effect (positioning vs floor accuracy)")
print(f"\nPhase 2 Figures (6 total):")
print(f"  5. TTS vs num_sweeps (different num_reads)")
print(f"  6. Success Rate vs num_reads")
print(f"  7. TTS vs beta (different gamma values)")
print(f"  8. Floor Accuracy vs TTS (colored by num_sweeps)")
print(f"  9. Mean 3D Error vs TTS (colored by success rate)")
print(f"  10. Pareto front with best configuration marked")
print(f"\nAll figures saved to: {output_dir}")
print(f"  Resolution: 300 DPI")
print(f"  Line thickness: 3.0")
print(f"  No titles (as requested)")
print("="*80)